# 0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 1. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [4]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
#             print(landmarks)
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
len(landmarks)

33

In [6]:
# landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

In [7]:
# landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

# 2. Calculate Angles

In [8]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def calculate_shoulder_angle(a,b):
    a = np.array(a)
    b = np.array(b)
    
    radians = np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [9]:
import math as m
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 - y1) * (-y1) / (m.sqrt(
        (x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / m.pi) * theta
    return degree

In [10]:
left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

In [11]:
left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

In [12]:
print(findAngle(left_shoulder[0],left_shoulder[1],left_elbow[0],left_elbow[1]))
print(calculate_shoulder_angle(left_shoulder,left_elbow))

147.98920676556693
121.24286033059875


In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
stage = None
suggestion = None
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        h, w = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_shoulder_angle(left_shoulder,right_shoulder)
            angle1 = calculate_angle(left_shoulder,right_shoulder,right_elbow)
            angle2 = calculate_angle(right_shoulder,left_shoulder,left_elbow)
            
            # Visualize angle  
            cv2.putText(image, str(round(angle,3)), 
                           tuple(np.multiply(left_shoulder, [450, 450]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(angle1,3)), 
                           tuple(np.multiply(right_shoulder, [600, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(angle2,3)), 
                           tuple(np.multiply(left_shoulder, [600, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            shoulder_posture = angle >= 0 and angle <= 10
            left_arm_posture = angle1 >= 90 and angle1 <= 124
            right_arm_posture = angle2 >= 90 and angle2 <= 124
            if shoulder_posture and left_arm_posture and right_arm_posture:
                stage = "Good"
                suggestion = None
            if shoulder_posture and not(left_arm_posture and right_arm_posture):
                stage="Bad"
                suggestion = "adjust your arm posture"
            if not(shoulder_posture) and (left_arm_posture and right_arm_posture):
                stage = "Bad"
                suggestion = "adjust your shoulder posture"
            if not(shoulder_posture) and not(left_arm_posture and right_arm_posture):
                stage = "Bad"
                suggestion = "adjust your posture"
                       
        except:
            pass
        
         # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        cv2.putText(image, stage, 
                    (20,35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, suggestion, 
                    (20,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [16]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
stage = None
suggestion = None
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        h, w = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            neck_inclination = findAngle(left_shoulder[0],left_shoulder[1],left_ear[0],left_ear[1])
            torso_inclination = findAngle(left_hip[0],left_hip[1],left_shoulder[0],left_shoulder[1])
            
#             angle_text_string = 'Neck : ' + str(int(neck_inclination)) + '  Torso : ' + str(int(torso_inclination))
            # Visualize angle  
            cv2.putText(image, str(round(neck_inclination,3)), 
                           tuple(np.multiply(left_shoulder, [w,h]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(torso_inclination,3)), 
                           tuple(np.multiply(left_hip, [w, h]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            

            cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(left_ear[0]*w),int(left_ear[1]*h)), (245,66,230), 2)
            cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)),(int(left_shoulder[0]*w), int(left_shoulder[1]*h-100)), (245,66,230), 2)
            cv2.line(image, (int(left_hip[0]*w), int(left_hip[1]*h)), (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (245,66,230), 2)
            cv2.line(image, (int(left_hip[0]*w), int(left_hip[1]*h)), (int(left_hip[0]*w), int(left_hip[1]*h-100)), (245,66,230), 2)
            
            if neck_inclination < 40 and torso_inclination < 10:
                stage = "Good"
                suggestion = None
            elif neck_inclination > 40 and torso_inclination < 10:
                stage = "Bad"
                suggestion = "Adjust your neck posture" 
                cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(left_ear[0]*w),int(left_ear[1]*h)), (255,0,0), 2)
                cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)),(int(left_shoulder[0]*w), int(left_shoulder[1]*h-100)), (255,0,0), 2)
            elif torso_inclination > 10 and neck_inclination < 40:
                stage = "Bad"
                suggestion = "Adjust your torso posture"
                cv2.line(image, (int(left_hip[0]*w), int(left_hip[1]*h)), (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (255,0,0), 2)
                cv2.line(image, (int(left_hip[0]*w), int(left_hip[1]*h)), (int(left_hip[0]*w), int(left_hip[1]*h-100)), (255,0,0), 2)
            
                
        except:
            pass
        
         # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        cv2.putText(image, stage, 
                    (20,35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, suggestion, 
                    (20,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
stage = None
suggestion = None
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        h, w = frame.shape[:2]

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            angle = calculate_shoulder_angle(left_shoulder,right_shoulder)
            angle1 = calculate_angle(left_shoulder,right_shoulder,right_elbow)
            angle2 = calculate_angle(right_shoulder,left_shoulder,left_elbow)
            
            
            # Visualize angle  
            cv2.putText(image, str(round(angle,3)), 
                           tuple(np.multiply(left_shoulder, [450, 450]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(angle1,3)), 
                           tuple(np.multiply(right_shoulder, [600, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(angle2,3)), 
                           tuple(np.multiply(left_shoulder, [600, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            #drawing joints
            cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (127, 255, 0), 2)
            cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(left_elbow[0]*w),int(left_elbow[1]*h)), (127, 255, 0), 2)
            cv2.line(image, (int(right_shoulder[0]*w),int(right_shoulder[1]*h)),(int(right_elbow[0]*w), int(right_elbow[1]*h)), (127, 255, 0), 2)
            
            shoulder_posture = angle >= 0 and angle <= 10
            left_arm_posture = angle1 >= 90 and angle1 <= 124
            right_arm_posture = angle2 >= 90 and angle2 <= 124
            if shoulder_posture and left_arm_posture and right_arm_posture:
                stage = "Good"
                suggestion = None
            if shoulder_posture and not(left_arm_posture and right_arm_posture):
                if(left_arm_posture):
                    cv2.line(image, (int(left_elbow[0]*w), int(left_elbow[1]*h)), (int(left_shoulder[0]*w),int(left_shoulder[1]*h)), (50, 50, 255), 2)
                if(right_arm_posture):
                    cv2.line(image, (int(right_elbow[0]*w), int(right_elbow[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (50, 50, 255), 2)
                if(not(left_arm_posture or right_arm_posture)):
                    cv2.line(image, (int(left_elbow[0]*w), int(left_elbow[1]*h)), (int(left_shoulder[0]*w),int(left_shoulder[1]*h)), (50, 50, 255), 2)
                    cv2.line(image, (int(right_elbow[0]*w), int(right_elbow[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (50, 50, 255), 2)
                stage="Bad"
                suggestion = "adjust your arm posture"
            if not(shoulder_posture) and (left_arm_posture and right_arm_posture):
                cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (50, 50, 255), 2)
                stage = "Bad"
                suggestion = "adjust your shoulder posture"
            if not(shoulder_posture) and not(left_arm_posture and right_arm_posture):
                stage = "Bad"
                suggestion = "adjust your posture"
 

            
        except:
            pass
        
         # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        cv2.putText(image, stage, 
                    (20,35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, suggestion, 
                    (20,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        
            
        # Render detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [17]:
def calculate_distance(x,y):
    return m.sqrt((x[0]-y[0])*2 + (x[1]-y[1])*2)*100
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
stage = None
suggestion = None
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        h, w = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
                        
            # Calculate angle
            distance = calculate_distance(left_ear,right_ear)
            
            if distance < 50:
                stage = "Neck"
                suggestion = None
            else:
                stage = "Good"
                suggestion = None
                       
        except:
            pass
        
         # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        cv2.putText(image, stage, 
                    (20,35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, suggestion, 
                    (20,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()